In [44]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

In [45]:
PROBLEM_SIZE = 5
NUM_SETS = 10
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

In [46]:
def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

In [47]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [48]:
frontier = LifoQueue()
frontier.put(State(set(), set(range(NUM_SETS))))

counter = 0
current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put(new_state)
    current_state = frontier.get()

print(f"Solved in {counter:,} steps")

Solved in 5 steps


In [49]:
print(current_state)

State(taken={5, 6, 7, 8, 9}, not_taken={0, 1, 2, 3, 4})


In [50]:
def key(state):
    indices = state.taken
    if len(indices) != 0:
        return sum([len(SETS[i]) for i in indices])
    else:
        return 0


In [51]:
frontier = PriorityQueue()
init = State(set(), set(range(NUM_SETS)))
frontier.put((key(init), init))

counter = 0
k, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put((key(new_state),new_state))
    k, current_state = frontier.get()


print(f"Solved in {counter:,} steps")

Solved in 43 steps


In [52]:
print(current_state)

State(taken={5, 7}, not_taken={0, 1, 2, 3, 4, 6, 8, 9})
